In [124]:
#подключаем нужные библиотеки
import numpy as np
import math
from matplotlib import pyplot as plt
from numpy import array
from pprint import pprint 
import random as random
from scipy import *
from numpy.linalg import *


In [125]:
#извлекаем данные из файлы
dataknown = np.genfromtxt("dataknown.txt", delimiter=' ', dtype=np.float)
alldata = np.genfromtxt("alldata.txt", delimiter=' ', dtype=np.float)

In [126]:
#Классифицируем по взлетной массе БЛА:
   # 1 класс: от 1000 до 1650 кг
   # 2 класс: от 2400 до 6500 кг
   # 3 класс: от 5300 до 10000 кг
   # 4 класс: свыше 10000 кг
    
#Физический смысл координат
    #1 - Отношение массы полезной нагрузки к взлетной массе БЛА
    #2 - Продолжительность полета (ч)
    #3 - Дальность полета (км)
    #4 - Крейсерская скорость полета (км/ч)
    #5 - Высота полета (практический потолок) (м)

In [127]:
D = []
X_star = np.eye(5)

alpha = 0.01
otn = []
number = []
i = 0
otn_pogr = 0
P = [0 for i in range(100)]
a=[]
b=[]
c=[]

In [128]:
for z in range (1,12): #количество точек, в которых знаем значение функции
    #print("z=",z)
    X = [[0 for i in range(10)] for j in range(z)]
    Diag = [[0 for i in range(5)] for j in range(5)]
    
    X[z-1] = [dataknown[(z-1),0], dataknown[(z-1),1],dataknown[(z-1),2], dataknown[(z-1),3], dataknown[(z-1),4]]\
    #считываем известные данные из файла
    
    P[0] = 95.1 #задаем значение функции в точке
    P[1] = 62.56
    P[2] = 72.44
    P[3] = 82.48
    P[4] = 0.5*20+0.02*25+0.48*30
    P[5] = 0.5*100+0.02*100+0.48*95
    P[6] = 0.5*50+0.02*55+0.48*60
    P[7] = 0.5*100+0.02*90+0.48*85
    P[8] = 0.5*50+0.02*55+0.48*55
    P[9] = 0.5*100+0.02*100+0.48*95
    P[10] = 0.5*20+0.02*25+0.48*20
    
    Y = P[0:z] #вектор из значений функции в точках
    D.append(X[z-1]) #складываем все сгенерированные точки в массив
    


In [129]:
for z in range (0,66):
    X_star = [alldata[z,0], alldata[z,1], alldata[z,2], alldata[z,3], alldata[z,4]]
    
    #блок подсчета весов
    weight = [0 for i in range (5)]
    weight_wave = [0 for i in range (5)]
    W_wave = [[0 for i in range(11)] for j in range(11)]
    W_wave0 = [[0 for i in range(11)] for j in range(11)]
    W_wave1 = [[0 for i in range(11)] for j in range(11)]
    W_wave2 = [[0 for i in range(11)] for j in range(11)] 
    W_wave3 = [[0 for i in range(11)] for j in range(11)]
    W_wave4 = [[0 for i in range(11)] for j in range(11)]
  
    for i in range(11):
        for j in range(11):
            W_wave[i][j] = (D[i][0] - X_star[0])*(D[j][0]-X_star[0]) + \
            (D[i][1] - X_star[1])*(D[j][1]-X_star[1]) + (D[i][2] - X_star[2])*(D[j][2]-X_star[2]) + \
            (D[i][3] - X_star[3])*(D[j][3]-X_star[3]) + (D[i][4] - X_star[4])*(D[j][4]-X_star[4]) 
           
            
    for i in range(11):
        for j in range(11):
            W_wave0[i][j] = (D[i][1] - X_star[1])*(D[j][1]-X_star[1]) + (D[i][2] - X_star[2])*(D[j][2]-X_star[2]) + \
            (D[i][3] - X_star[3])*(D[j][3]-X_star[3]) + (D[i][4] - X_star[4])*(D[j][4]-X_star[4]) 
            
            
            W_wave1[i][j] = (D[i][0] - X_star[0])*(D[j][0]-X_star[0]) + (D[i][2] - X_star[2])*(D[j][2]-X_star[2]) + \
            (D[i][3] - X_star[3])*(D[j][3]-X_star[3]) + (D[i][4] - X_star[4])*(D[j][4]-X_star[4]) 
            
            W_wave2[i][j] = (D[i][0] - X_star[0])*(D[j][0]-X_star[0]) + (D[i][1] - X_star[1])*(D[j][1]-X_star[1]) + \
            (D[i][3] - X_star[3])*(D[j][3]-X_star[3]) + (D[i][4] - X_star[4])*(D[j][4]-X_star[4]) 
            
            W_wave3[i][j] = (D[i][0] - X_star[0])*(D[j][0]-X_star[0]) + (D[i][1] - X_star[1])*(D[j][1]-X_star[1]) + \
            (D[i][2] - X_star[2])*(D[j][2]-X_star[2]) + (D[i][4] - X_star[4])*(D[j][4]-X_star[4]) 
            
            W_wave4[i][j] = (D[i][0] - X_star[0])*(D[j][0]-X_star[0]) + (D[i][1] - X_star[1])*(D[j][1]-X_star[1]) + \
            (D[i][2] - X_star[2])*(D[j][2]-X_star[2]) + (D[i][3] - X_star[3])*(D[j][3]-X_star[3])
            
    E1 = [1 for g in range (11)] 
    Y_star0 = (np.dot(np.dot(linalg.inv(W_wave0+ np.eye(11)*0.01),E1),Y))/(np.dot(np.dot(linalg.inv(W_wave0+ np.eye(11)*0.01),E1),E1))
    Y_star1 = (np.dot(np.dot(linalg.inv(W_wave1+ np.eye(11)*0.01),E1),Y))/(np.dot(np.dot(linalg.inv(W_wave1+ np.eye(11)*0.01),E1),E1))
    Y_star2 = (np.dot(np.dot(linalg.inv(W_wave2+ np.eye(11)*0.01),E1),Y))/(np.dot(np.dot(linalg.inv(W_wave2+ np.eye(11)*0.01),E1),E1)) 
    Y_star3 = (np.dot(np.dot(linalg.inv(W_wave3+ np.eye(11)*0.01),E1),Y))/(np.dot(np.dot(linalg.inv(W_wave3+ np.eye(11)*0.01),E1),E1))
    Y_star4 = (np.dot(np.dot(linalg.inv(W_wave4+ np.eye(11)*0.01),E1),Y))/(np.dot(np.dot(linalg.inv(W_wave4+ np.eye(11)*0.01),E1),E1))
    
    Y_star_wave = (np.dot(np.dot(linalg.inv(W_wave + np.eye(11)*0.01),E1),Y))/(np.dot(np.dot(linalg.inv(W_wave+ np.eye(11)*0.01),E1),E1)) 
    
    weight_wave[0] = (Y_star0 - Y_star_wave)**2
    weight_wave[1] = (Y_star1 - Y_star_wave)**2
    weight_wave[2] = (Y_star2 - Y_star_wave)**2        
    weight_wave[3] = (Y_star3 - Y_star_wave)**2
    weight_wave[4] = (Y_star4 - Y_star_wave)**2
  
    
    for k in range(5):   
        weight[k] = 5*weight_wave[k]/(weight_wave[0] + weight_wave[1]+ weight_wave[2]+ weight_wave[3] +\
                                      weight_wave[4])
    for i in range(5):
        for j in range(5):
            Diag[i][i]=weight[i]
        
    X_matrix = [[0 for i in range(5)] for j in range(11)]
    X_matrix_t = [[0 for i in range(11)] for i in range(5)]
    
    for j in range (11):
        for i in range (5):
            X_matrix[j][i] = D[j][i] - X_star[i]
    
    X_matrix_t = list(zip(*X_matrix)) #транспонирование
       
    W = np.dot (np.dot(X_matrix,Diag), X_matrix_t) #перемножаем транспонированную на обычную
    
    for i in range (11):
        W[i][i] += alpha #к диагональным элементам прибавляем альфу
    V = [[0 for i in range (11)] for j in range (11)]    
    V = linalg.inv(W) # обратная матрица
    
    E = [1 for g in range (11)] #единичный вектор 
    V1 = np.dot(V, E)
    V2 = np.dot(V1,E)
    V3 = np.dot(V1, Y)
    print(round(V3/V2,2))
   
    X_matrix = [[0 for i in range(5)] for j in range(z)]
    #print("___________________________")
   
    

37.25
58.97
60.59
75.28
43.56
33.38
55.05
66.66
51.53
82.37
78.27
47.9
58.22
62.7
67.42
76.28
76.42
42.39
52.21
73.17
72.29
74.52
49.39
74.69
59.81
52.61
45.52
64.57
47.24
39.62
44.12
33.14
83.39
52.09
83.35
35.67
26.02
57.61
69.37
97.27
87.35
88.05
78.45
35.66
33.56
72.49
97.34
33.63
63.88
55.05
41.12
52.32
53.32
88.68
52.17
69.53
64.65
67.8
67.1
35.82
36.06
30.99
41.69
48.62
51.74
85.93
[]
[37.25, 58.97, 60.59, 75.28, 43.56, 33.38, 55.05, 66.66, 51.53, 82.37, 78.27, 47.9, 58.22, 62.7, 67.42, 76.28, 76.42, 42.39, 52.21, 73.17, 72.29, 74.52, 49.39, 74.69, 59.81, 52.61, 45.52, 64.57, 47.24, 39.62, 44.12, 33.14, 83.39, 52.09, 83.35, 35.67, 26.02, 57.61, 69.37, 97.27, 87.35, 88.05, 78.45, 35.66, 33.56, 72.49, 97.34, 33.63, 63.88, 55.05, 41.12, 52.32, 53.32, 88.68, 52.17, 69.53, 64.65, 67.8, 67.1, 35.82, 36.06, 30.99, 41.69, 48.62, 51.74, 85.93]


In [121]:
a = 1/19 *((35.45-37.33)**2+(57.94-58.89)**2+ (59.12-60.56)**2+ (74.23-75.37)**2+(45.27-43.78)**2+ (32.14-33.52)**2+ (56.53-55.24)**2+ (66.96-66.61)**2 +(53.08-51.58)**2+(100.00-95.00)**2+ (81.13-78.16)**2+(65.00-62.67)**2+(59.24-58.33)**2+ (70.00-72.33)**2+(67.53-67.47)**2+ (76.13-76.53)**2+ (80.00-83.00)**2+ (40.56-42.91)**2+ (53.54-52.44)**2)
a

4.072321052631578

In [122]:
1/19 *((37.51-37.33)**2+(58.73-58.89)**2+ (60.47-60.56)**2+ (75.59-75.37)**2+(44.24-43.78)**2+ (33.77-33.52)**2+ (55.65-55.24)**2+ (66.57-66.61)**2+ (51.71-51.58)**2+(95.00-95.00)**2+ (77.91-78.16)**2+ (63.00-62.67)**2+(58.54-58.33)**2+ (72.00-72.33)**2+(67.58-67.47)**2+ (77.10-76.53)**2+ (84.00-83.00)**2+ (44.01-42.91)**2+ (52.93-52.44)**2 )

0.1940368421052629

In [123]:
1/19 *((39.04-37.33)**2+(59.99-58.89)**2+ (62.09-60.56)**2+ (76.29-75.37)**2+(41.83-43.78)**2+ (34.66-33.52)**2+ (53.54-55.24)**2+ (66.31-66.61)**2 + (49.94-51.58)**2+(90.00-95.00)**2+ (75.44-78.16)**2+(60.00-62.67)**2+(57.21-58.33)**2+ (75.00-72.33)**2+(67.31-67.47)**2+ (76.36-76.53)**2+ (85.00-83.00)**2+ (44.17-42.91)**2+ (50.86-52.44)**2 )

3.9022210526315813